In [1]:
from dotenv import load_dotenv

env_path = '/teamspace/studios/this_studio/Deeplab/credentials.env'
load_dotenv(dotenv_path=env_path)
import boto3

%cd /teamspace/studios/this_studio/Deeplab/
from inference.infer import *
from post_processing.control_random_field import *


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/teamspace/studios/this_studio/Deeplab


In [2]:
bucket = get_s3_bucket('treetracker-training-images')
print(type(bucket))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import matplotlib.pyplot as plt
img_link = 'https://treetracker-production-images.s3.eu-central-1.amazonaws.com/2021.03.28.12.34.59_-4.531118333333334_38.248731666666664_6973c014-d0e7-4fa8-a80f-9495bfc8af3d_IMG_20210322_094829_8180242403936928723.jpg'
inhouse_default = '/teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth'
downstreamed_large = '/teamspace/studios/this_studio/Deeplab/saved_models/india_sam_dino_250_samples_0.8495187035894669.pth'
downstreamed_small = '/teamspace/studios/this_studio/Deeplab/saved_models/india_sam_dino_250_samples_0.8940880817269634.pth'
downstreamed_tuned = '/teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8213798905479425.pth'
crf_large = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8385291022474306.pth'
crf_proper = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth'
crf_small = '/teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth'
crf_sam97 = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_970.9094360785270311.pth'
crf_sam95 = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth'
check_kl = '/teamspace/studios/this_studio/Deeplab/saved_models/kl_check.pth'
check_kl_2 = '/teamspace/studios/this_studio/Deeplab/saved_models/kl_check_temp_2_weighted.pth'
crf_sam95_combined_loss = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_with_prob_0.8744961851984793.pth'
crf_sam95_dice = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_without_prob0.8729896976681869.pth'
crf_sam95_dice_fit = '/teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_without_prob0.8504620350701382.pth'

<class 'boto3.resources.factory.s3.Bucket'>


In [3]:
def get_target(mask_path):
    mask = Image.open(mask_path)
    mask_array = np.array(mask)
    mask_array = (mask_array > 128).astype(np.uint8)
    mask_array = mask_array * 255
    mask = Image.fromarray(mask_array.astype(np.uint8))
    mask = mask_transform(mask)
    mask = torch.squeeze(mask, 0)
    mask = mask.to(device, dtype=torch.long)
    mask = mask.float()
    return mask.cpu().numpy()

def decode_target_single_channel(mask):
    leaf_color = [255]
    rgb_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
    rgb_mask[mask == 1] = leaf_color
    return Image.fromarray(rgb_mask)

def polygon_vertices(mask):
    '''
    Find contours in the binary image and return both external and internal polygons.

    mask: binary image to find contours in
    '''
    # Convert the image into 0 and 1 (background and foreground)
    _, binary_image = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # Find contours and hierarchy
    contours, hierarchy = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Extract segmentations
    external_polygons = []
    internal_polygons = []
    if hierarchy is not None:
        hierarchy = hierarchy[0]  # Flatten the hierarchy array
        for i, contour in enumerate(contours):
            # Approximate contour to polygon
            if len(contour) < 3:
                continue

            contour = cv2.approxPolyDP(contour, 1, True)
            segmentation = contour.flatten().tolist()

            # Determine if the contour is external or internal
            if hierarchy[i][3] == -1:
                # External contour
                external_polygons.append(segmentation)
            else:
                # Internal contour
                internal_polygons.append(segmentation)

    return external_polygons, internal_polygons




def calculate_polygon_area(vertices):
    # Ensure the polygon is closed (first and last vertices are the same)
    if vertices[0] != vertices[-2] or vertices[1] != vertices[-1]:
        vertices.extend(vertices[:2])

    # Calculate the area using the shoelace formula
    area = 0
    n = len(vertices) // 2
    for i in range(n - 1):
        area += (vertices[2*i] * vertices[2*i+3] - vertices[2*i+1] * vertices[2*i+2])
    area += (vertices[2*n-2] * vertices[1] - vertices[2*n-1] * vertices[0])
    return abs(area) / 2

def create_pair(t):
  pair = list(zip(t[::2],t[1::2]))
  new_pair = []
  for i in pair:
    new_pair.append(list(i))
  return new_pair

def remove_small_polygons(mask, threshold):
    if not isinstance(mask,np.ndarray):
        print('Takes a numpy array as input with entries either 0 or 255.')
        return
    segmentation, inner_polygons = polygon_vertices(mask)[0], polygon_vertices(mask)[1]
      
    outer_polygons = [polygon for polygon in segmentation if calculate_polygon_area(polygon) > threshold]
    
    binary_mask = np.zeros(mask.shape, dtype=np.uint8)
    for i in outer_polygons:
      pair = create_pair(i)
      polygon_points = np.array(pair, np.int32)
      cv2.fillPoly(binary_mask, [polygon_points], 255)
    
    for i in inner_polygons:
        pair = create_pair(i)
        polygon_points = np.array(pair, np.int32)
        cv2.fillPoly(binary_mask, [polygon_points], 0)

    
    return binary_mask/255

def metrics_from_dataset(checkpoint, dataset_path):
    metrics = BinarySegMetrics()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_deeplab_model(checkpoint, device).eval()
    sample_dir = os.path.join(dataset_path, 'samples')
    mask_dir = os.path.join(dataset_path, 'binary_masks')
    
    file_list = os.listdir(sample_dir)
    
    for i in tqdm(file_list, desc="Processing images"):
        img_path = os.path.join(sample_dir, i)
        ext = os.path.basename(img_path).split('.')[-1]
        img_name = os.path.basename(img_path)[:-len(ext) - 1]
        mask_path = os.path.join(mask_dir, img_name + '_binarymask.' + ext)
        
        target = get_target(mask_path)
        img = Image.open(img_path)
        
        img_transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        img_tensor = img_transform(img).unsqueeze(0).to(device, dtype=torch.float32)
    
        with torch.no_grad():
            output = model(img_tensor)
            output = torch.squeeze(output, dim=1)
            prob = torch.sigmoid(output).detach()
            pred = (prob > 0.5).long().cpu().numpy()[0]
            
        metrics.update(target, pred)
        
        # The following lines are commented out as they appear to be part of additional processing
        # denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        # img_np = img_tensor[0].detach().cpu().numpy()
        # img_np = (denorm(img_np) * 255).transpose(1, 2, 0).astype(np.uint8)
        # cleaned_mask = apply_dense_crf(img_np, np.array(decode_target(pred)))
        # prob_np = prob[0].cpu().numpy()
        # count = np.sum(prob_np > 0.5)
        # confidence = np.sum(prob_np[prob_np > 0.5]) / count if count != 0 else 0
    # Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth
    # Deeplab/india_sam_dino_annotations Deeplab/india_sam_dino_annotations_large
        
    return metrics.get_results()

def metrics_from_dataset_crf(checkpoint, dataset_path, with_prob = True):
    metrics = BinarySegMetrics()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_deeplab_model(checkpoint, device).eval()
    sample_dir = os.path.join(dataset_path, 'samples')
    mask_dir = os.path.join(dataset_path, 'binary_masks')
    
    file_list = os.listdir(sample_dir)
    
    for i in tqdm(file_list, desc="Processing images"):
        img_path = os.path.join(sample_dir, i)
        ext = os.path.basename(img_path).split('.')[-1]
        img_name = os.path.basename(img_path)[:-len(ext) - 1]
        mask_path = os.path.join(mask_dir, img_name + '_binarymask.' + ext)
        
        target = get_target(mask_path)
        img = Image.open(img_path)
        
        img_transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        img_tensor = img_transform(img).unsqueeze(0).to(device, dtype=torch.float32)
    
        with torch.no_grad():
            output = model(img_tensor)
            output = torch.squeeze(output, dim=1)
            prob = torch.sigmoid(output).detach()
            
            pred = (prob > 0.5).long().cpu().numpy()[0]
            
        denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        img_np = img_tensor[0].detach().cpu().numpy() 
        img_np = (denorm(img_np) * 255).transpose(1, 2, 0).astype(np.uint8)
        
        if with_prob:
            prob_np = prob[0].cpu().numpy()
            cleaned_mask = crf_with_prob(img_np,np.array(decode_target_single_channel(pred)),prob_np)
        else:
            cleaned_mask = apply_dense_crf(img_np,np.array(decode_target_single_channel(pred)))
        '''    
        cleaned_mask = remove_small_polygons((cleaned_mask * 255).astype(np.uint8),100)
        cleaned_mask = cleaned_mask.astype(np.int64)
        '''
        metrics.update(target, cleaned_mask)
        
        # The following lines are commented out as they appear to be part of additional processing
        # denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        # img_np = img_tensor[0].detach().cpu().numpy()
        # img_np = (denorm(img_np) * 255).transpose(1, 2, 0).astype(np.uint8)
        # cleaned_mask = apply_dense_crf(img_np, np.array(decode_target(pred)))
        # prob_np = prob[0].cpu().numpy()
        # count = np.sum(prob_np > 0.5)
        # confidence = np.sum(prob_np[prob_np > 0.5]) / count if count != 0 else 0
        
    return metrics.get_results()

def metrics_from_dataset_poly_remove(checkpoint, dataset_path):
    metrics = BinarySegMetrics()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_deeplab_model(checkpoint, device).eval()
    sample_dir = os.path.join(dataset_path, 'samples')
    mask_dir = os.path.join(dataset_path, 'binary_masks')
    
    file_list = os.listdir(sample_dir)
    
    for i in tqdm(file_list, desc="Processing images"):
        img_path = os.path.join(sample_dir, i)
        ext = os.path.basename(img_path).split('.')[-1]
        img_name = os.path.basename(img_path)[:-len(ext) - 1]
        mask_path = os.path.join(mask_dir, img_name + '_binarymask.' + ext)
        
        target = get_target(mask_path)
        img = Image.open(img_path)
        
        img_transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        img_tensor = img_transform(img).unsqueeze(0).to(device, dtype=torch.float32)
    
        with torch.no_grad():
            output = model(img_tensor)
            output = torch.squeeze(output, dim=1)
            prob = torch.sigmoid(output).detach()
            pred = (prob > 0.5).long().cpu().numpy()[0]
            
        denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        img_np = img_tensor[0].detach().cpu().numpy() 
        img_np = (denorm(img_np) * 255).transpose(1, 2, 0).astype(np.uint8)
        #threshold_value = 127 
        #image_array_gray = np.array(pred*255)
        #binary_image_array = np.where(image_array_gray > threshold_value, 255, 0).astype(np.uint8)
        
        cleaned_mask = remove_small_polygons((pred * 255).astype(np.uint8),100)
        
        #print(type(cleaned_mask))
        #print(type(target))
        cleaned_mask = cleaned_mask.astype(np.int64)
        metrics.update(target, cleaned_mask)
        
        # The following lines are commented out as they appear to be part of additional processing
        # denorm = utils.Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        # img_np = img_tensor[0].detach().cpu().numpy()
        # img_np = (denorm(img_np) * 255).transpose(1, 2, 0).astype(np.uint8)
        # cleaned_mask = apply_dense_crf(img_np, np.array(decode_target(pred)))
        # prob_np = prob[0].cpu().numpy()
        # count = np.sum(prob_np > 0.5)
        # confidence = np.sum(prob_np[prob_np > 0.5]) / count if count != 0 else 0
        
    return metrics.get_results()


In [4]:
metrics_from_dataset(crf_sam95_dice,'/teamspace/studios/this_studio/Deeplab/inhouse_india')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_without_prob0.8729896976681869.pth


Processing images: 100%|██████████| 9/9 [00:00<00:00, 14.15it/s]


{'Foreground Acc': 0.9580378303715622,
 'Precision': 0.698475250120145,
 'Recall': 0.9580378303715622,
 'F1 Score': 0.8079208321068104,
 'IoU Foreground': 0.6777409201225134,
 'IoU Background': 0.9254555421103107,
 'Mean IoU': 0.801598231116412}

In [11]:
metrics_from_dataset(crf_sam95,'/teamspace/studios/this_studio/Deeplab/inhouse_india')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 9/9 [00:00<00:00, 29.63it/s]


{'Foreground Acc': 0.9293219836226191,
 'Precision': 0.7946402951424473,
 'Recall': 0.9293219836226191,
 'F1 Score': 0.8567202478203164,
 'IoU Foreground': 0.7493531186806762,
 'IoU Background': 0.9493712459714649,
 'Mean IoU': 0.8493621823260706}

In [5]:
metrics_from_dataset(crf_sam95_dice,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_without_prob0.8729896976681869.pth


Processing images: 100%|██████████| 250/250 [00:04<00:00, 51.26it/s]


{'Foreground Acc': 0.9026589771863649,
 'Precision': 0.9010724277115912,
 'Recall': 0.9026589771863649,
 'F1 Score': 0.9018650046900909,
 'IoU Foreground': 0.8212697059486497,
 'IoU Background': 0.945922799729736,
 'Mean IoU': 0.8835962528391929}

In [6]:
metrics_from_dataset(crf_sam95_dice,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_95_without_prob0.8729896976681869.pth


Processing images: 100%|██████████| 4071/4071 [01:37<00:00, 41.56it/s]


{'Foreground Acc': 0.8806641354884861,
 'Precision': 0.8774461484391766,
 'Recall': 0.8806641354884861,
 'F1 Score': 0.8790521969151777,
 'IoU Foreground': 0.7842043978283791,
 'IoU Background': 0.9470409705576935,
 'Mean IoU': 0.8656226841930363}

In [10]:
# check_kl_2
#india_sam_dino_annotations_large

metrics_from_dataset(check_kl_2,'/teamspace/studios/this_studio/Deeplab/crf_sam_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/kl_check_temp_2.pth


Processing images: 100%|██████████| 250/250 [00:08<00:00, 29.61it/s]


{'Foreground Acc': 0.9591152605056845,
 'Precision': 0.9341784923971478,
 'Recall': 0.9591152605056845,
 'F1 Score': 0.9464826540736827,
 'IoU Foreground': 0.8984025348357952,
 'IoU Background': 0.9750410299888329,
 'Mean IoU': 0.9367217824123141}

In [5]:
#india_sam_dino_annotations_large

metrics_from_dataset(check_kl,'/teamspace/studios/this_studio/Deeplab/crf_sam_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/kl_check.pth


Processing images: 100%|██████████| 250/250 [00:23<00:00, 10.75it/s]


{'Foreground Acc': 0.9671239895597793,
 'Precision': 0.8964184870336949,
 'Recall': 0.9671239895597793,
 'F1 Score': 0.9304299036745703,
 'IoU Foreground': 0.8699101693952707,
 'IoU Background': 0.9666576534564355,
 'Mean IoU': 0.9182839114258531}

In [6]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(check_kl,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/kl_check.pth


Processing images: 100%|██████████| 250/250 [02:10<00:00,  1.92it/s]


{'Foreground Acc': 0.8473926829339183,
 'Precision': 0.9145589006121304,
 'Recall': 0.8473926829339183,
 'F1 Score': 0.8796955917836133,
 'IoU Foreground': 0.7852290728590083,
 'IoU Background': 0.9371530171018962,
 'Mean IoU': 0.8611910449804523}

In [ ]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(check_kl_2,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/kl_check_temp_2.pth


Processing images: 100%|█████████▉| 249/250 [01:49<00:00,  2.62it/s]

In [ ]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(crf_small,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

In [4]:
#india_sam_dino_annotations_large

metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 244/244 [01:11<00:00,  3.42it/s]


{'Foreground Acc': 0.9302469170876019,
 'Precision': 0.9198224517006076,
 'Recall': 0.9302469170876019,
 'F1 Score': 0.9250053153659921,
 'IoU Foreground': 0.860474315443633,
 'IoU Background': 0.9723015798083208,
 'Mean IoU': 0.9163879476259769}

In [10]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam95,'/teamspace/studios/this_studio/Deeplab/inhouse_india')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 9/9 [00:02<00:00,  3.60it/s]


{'Foreground Acc': 0.9293219836226191,
 'Precision': 0.7946402951424473,
 'Recall': 0.9293219836226191,
 'F1 Score': 0.8567202478203164,
 'IoU Foreground': 0.7493531186806762,
 'IoU Background': 0.9493712459714649,
 'Mean IoU': 0.8493621823260706}

In [6]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(crf_sam95,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 244/244 [02:55<00:00,  1.39it/s]


{'Foreground Acc': 0.9246824282989672,
 'Precision': 0.8602101558934871,
 'Recall': 0.9246824282989672,
 'F1 Score': 0.8912818875976206,
 'IoU Foreground': 0.8038850250821499,
 'IoU Background': 0.9586184839946985,
 'Mean IoU': 0.8812517545384242}

In [7]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(crf_sam97,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_970.9094360785270311.pth


Processing images: 100%|██████████| 244/244 [02:51<00:00,  1.43it/s]


{'Foreground Acc': 0.9191484424226131,
 'Precision': 0.862738390714087,
 'Recall': 0.9191484424226131,
 'F1 Score': 0.890050516448493,
 'IoU Foreground': 0.8018838060995328,
 'IoU Background': 0.9583849417186635,
 'Mean IoU': 0.8801343739090981}

In [5]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(crf_sam95,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 250/250 [03:05<00:00,  1.35it/s]


{'Foreground Acc': 0.8625361222924111,
 'Precision': 0.9365772367840469,
 'Recall': 0.8625361222924111,
 'F1 Score': 0.8980331272263341,
 'IoU Foreground': 0.8149365914838914,
 'IoU Background': 0.9466687669515649,
 'Mean IoU': 0.8808026792177281}

In [6]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam95,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 4071/4071 [02:14<00:00, 30.22it/s]


{'Foreground Acc': 0.8752205669366061,
 'Precision': 0.8957124782256072,
 'Recall': 0.8752205669366061,
 'F1 Score': 0.885347964053605,
 'IoU Foreground': 0.7942819243154218,
 'IoU Background': 0.9505216337747832,
 'Mean IoU': 0.8724017790451024}

In [12]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam97,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_970.9094360785270311.pth


Processing images: 100%|██████████| 4071/4071 [02:06<00:00, 32.28it/s]


{'Foreground Acc': 0.8641435446844463,
 'Precision': 0.8998177671089699,
 'Recall': 0.8641435446844463,
 'F1 Score': 0.8816199194856866,
 'IoU Foreground': 0.7883008065381966,
 'IoU Background': 0.9494682419094648,
 'Mean IoU': 0.8688845242238307}

In [13]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam97,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_970.9094360785270311.pth


Processing images: 100%|██████████| 250/250 [00:08<00:00, 28.55it/s]


{'Foreground Acc': 0.8886354517281331,
 'Precision': 0.9231844538351912,
 'Recall': 0.8886354517281331,
 'F1 Score': 0.9055805509622739,
 'IoU Foreground': 0.8274529036910938,
 'IoU Background': 0.9491848983691515,
 'Mean IoU': 0.8883189010301227}

In [14]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam97,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_970.9094360785270311.pth


Processing images: 100%|██████████| 244/244 [00:08<00:00, 30.16it/s]


{'Foreground Acc': 0.9330814949098515,
 'Precision': 0.8381518298509112,
 'Recall': 0.9330814949098515,
 'F1 Score': 0.8830727735594605,
 'IoU Foreground': 0.790626956398642,
 'IoU Background': 0.9546021434648322,
 'Mean IoU': 0.872614549931737}

In [9]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam95,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 250/250 [00:11<00:00, 21.69it/s]


{'Foreground Acc': 0.8976546928219353,
 'Precision': 0.9087204773517745,
 'Recall': 0.8976546928219353,
 'F1 Score': 0.9031536908025217,
 'IoU Foreground': 0.8234095180238384,
 'IoU Background': 0.9470774242498207,
 'Mean IoU': 0.8852434711368296}

In [10]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 250/250 [00:11<00:00, 22.20it/s]


{'Foreground Acc': 0.8817153485158494,
 'Precision': 0.9143330718283333,
 'Recall': 0.8817153485158494,
 'F1 Score': 0.8977280277690897,
 'IoU Foreground': 0.8144342325534777,
 'IoU Background': 0.945014057291501,
 'Mean IoU': 0.8797241449224893}

In [7]:
#india_sam_dino_annotations_large

metrics_from_dataset(crf_sam95,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_sam_threshold_950.8867386284487256.pth


Processing images: 100%|██████████| 244/244 [00:10<00:00, 22.61it/s]


{'Foreground Acc': 0.9433194086541946,
 'Precision': 0.8247468709716301,
 'Recall': 0.9433194086541946,
 'F1 Score': 0.8800572009992784,
 'IoU Foreground': 0.7858054909451777,
 'IoU Background': 0.9526700196625988,
 'Mean IoU': 0.8692377553038882}

In [33]:
#india_sam_dino_annotations_large

metrics_from_dataset_crf(crf_proper,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 4071/4071 [50:57<00:00,  1.33it/s]  


{'Foreground Acc': 0.8214854066255953,
 'Precision': 0.9276452000142961,
 'Recall': 0.8214854066255953,
 'F1 Score': 0.8713437309314832,
 'IoU Foreground': 0.7720186870096469,
 'IoU Background': 0.9476631980324502,
 'Mean IoU': 0.8598409425210486}

In [8]:
# crf_proper
metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 244/244 [00:10<00:00, 22.85it/s]


{'Foreground Acc': 0.9353319319936574,
 'Precision': 0.8436093005257606,
 'Recall': 0.9353319319936574,
 'F1 Score': 0.8871059959536517,
 'IoU Foreground': 0.7971163405753302,
 'IoU Background': 0.9562440273502152,
 'Mean IoU': 0.8766801839627727}

In [6]:
metrics_from_dataset(crf_small,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 244/244 [00:07<00:00, 33.21it/s]


{'Foreground Acc': 0.913678452852416,
 'Precision': 0.8268516931492811,
 'Recall': 0.913678452852416,
 'F1 Score': 0.8680993862364268,
 'IoU Foreground': 0.7669395843421213,
 'IoU Background': 0.9491697852830989,
 'Mean IoU': 0.85805468481261}

In [4]:
# crf_proper
metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 244/244 [00:07<00:00, 31.87it/s]


{'Foreground Acc': 0.9353319319936574,
 'Precision': 0.8436093005257606,
 'Recall': 0.9353319319936574,
 'F1 Score': 0.8871059959536517,
 'IoU Foreground': 0.7971163405753302,
 'IoU Background': 0.9562440273502152,
 'Mean IoU': 0.8766801839627727}

In [31]:
# crf_proper
metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/inhouse_india')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 9/9 [00:02<00:00,  3.35it/s]


{'Foreground Acc': 0.9172262121988417,
 'Precision': 0.7557217234169138,
 'Recall': 0.9172262121988417,
 'F1 Score': 0.8286782380838946,
 'IoU Foreground': 0.7074727585768595,
 'IoU Background': 0.9383499361231346,
 'Mean IoU': 0.822911347349997}

In [32]:
# crf_proper
metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/inhouse_india')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 9/9 [00:03<00:00,  2.81it/s]


{'Foreground Acc': 0.8671202955479052,
 'Precision': 0.8537394095296533,
 'Recall': 0.8671202955479052,
 'F1 Score': 0.8603778296719223,
 'IoU Foreground': 0.7549676130153157,
 'IoU Background': 0.9546213039134409,
 'Mean IoU': 0.8547944584643783}

In [30]:
# crf_proper
metrics_from_dataset_crf(crf_proper,'/teamspace/studios/this_studio/Deeplab/inhouse_test')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 10/10 [00:08<00:00,  1.24it/s]


{'Foreground Acc': 0.9710045978154664,
 'Precision': 0.9603842733461212,
 'Recall': 0.9710045978154664,
 'F1 Score': 0.9656652360515021,
 'IoU Foreground': 0.9336099585062241,
 'IoU Background': 0.9729452622876146,
 'Mean IoU': 0.9532776103969194}

In [29]:
# crf_proper
metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/inhouse_test')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 10/10 [00:02<00:00,  3.73it/s]


{'Foreground Acc': 0.9359948067936386,
 'Precision': 0.9660963341314475,
 'Recall': 0.9359948067936386,
 'F1 Score': 0.9508073847288068,
 'IoU Foreground': 0.9062276753473374,
 'IoU Background': 0.962564947579602,
 'Mean IoU': 0.9343963114634697}

In [20]:
# crf_proper
metrics_from_dataset_poly_remove(crf_proper,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 244/244 [01:13<00:00,  3.31it/s]


{'Foreground Acc': 0.9337854542775799,
 'Precision': 0.8463102257658198,
 'Recall': 0.9337854542775799,
 'F1 Score': 0.8878985410573325,
 'IoU Foreground': 0.7983970652295551,
 'IoU Background': 0.9566740775257182,
 'Mean IoU': 0.8775355713776367}

In [6]:
# crf_proper
metrics_from_dataset_crf(crf_proper,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 244/244 [03:02<00:00,  1.34it/s]


{'Foreground Acc': 0.919296770309781,
 'Precision': 0.87008069133759,
 'Recall': 0.919296770309781,
 'F1 Score': 0.8940118969858526,
 'IoU Foreground': 0.8083377158844689,
 'IoU Background': 0.9600545171811911,
 'Mean IoU': 0.88419611653283}

In [22]:
# crf_proper
metrics_from_dataset_crf(crf_proper,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 244/244 [02:55<00:00,  1.39it/s]


{'Foreground Acc': 0.9154983602690995,
 'Precision': 0.8746326501595297,
 'Recall': 0.9154983602690995,
 'F1 Score': 0.8945990571574319,
 'IoU Foreground': 0.8092982577497597,
 'IoU Background': 0.9604934101313581,
 'Mean IoU': 0.8848958339405589}

In [4]:
# crf_proper
metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 250/250 [01:15<00:00,  3.31it/s]


{'Foreground Acc': 0.8817154177189576,
 'Precision': 0.9143331435915056,
 'Recall': 0.8817154177189576,
 'F1 Score': 0.8977280982289835,
 'IoU Foreground': 0.8144343485365152,
 'IoU Background': 0.9450140941323633,
 'Mean IoU': 0.8797242213344392}

In [5]:
# crf_proper
metrics_from_dataset_crf(crf_proper,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 250/250 [03:07<00:00,  1.34it/s]


{'Foreground Acc': 0.851119131905138,
 'Precision': 0.9361168016994763,
 'Recall': 0.851119131905138,
 'F1 Score': 0.8915968000009278,
 'IoU Foreground': 0.8043975333179066,
 'IoU Background': 0.9438236383195845,
 'Mean IoU': 0.8741105858187456}

In [7]:
metrics_from_dataset(crf_large,'/teamspace/studios/this_studio/Deeplab/pilot_annotations_eastafrica_haiti_freetown')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8385291022474306.pth


Processing images: 100%|██████████| 244/244 [00:07<00:00, 31.03it/s]


{'Foreground Acc': 0.9471578911482642,
 'Precision': 0.780042646625894,
 'Recall': 0.9471578911482642,
 'F1 Score': 0.855515653250101,
 'IoU Foreground': 0.7475118866235176,
 'IoU Background': 0.9410699980040913,
 'Mean IoU': 0.8442909423138045}

In [6]:
metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/crf_sam_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 250/250 [01:07<00:00,  3.73it/s]


{'Foreground Acc': 0.8420827179865543,
 'Precision': 0.9576826669045405,
 'Recall': 0.8420827179865543,
 'F1 Score': 0.8961701673847741,
 'IoU Foreground': 0.811873479865589,
 'IoU Background': 0.9562760729604315,
 'Mean IoU': 0.8840747764130102}

In [7]:
metrics_from_dataset(crf,'/teamspace/studios/this_studio/Deeplab/crf_sam_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 250/250 [01:06<00:00,  3.77it/s]


{'Foreground Acc': 0.9614520470659847,
 'Precision': 0.9339192895685574,
 'Recall': 0.9614520470659847,
 'F1 Score': 0.9474856935891636,
 'IoU Foreground': 0.9002117005137639,
 'IoU Background': 0.9754619443907816,
 'Mean IoU': 0.9378368224522727}

In [4]:
metrics_from_dataset(crf_small,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 250/250 [01:06<00:00,  3.74it/s]


{'Foreground Acc': 0.8296436081448737,
 'Precision': 0.9330430401909089,
 'Recall': 0.8296436081448737,
 'F1 Score': 0.8783106119846065,
 'IoU Foreground': 0.7830248029167884,
 'IoU Background': 0.9379614562174732,
 'Mean IoU': 0.8604931295671308}

In [6]:
metrics_from_dataset_crf(crf_small,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 250/250 [01:47<00:00,  2.34it/s]


{'Foreground Acc': 0.8067870671570491,
 'Precision': 0.9465185416001765,
 'Recall': 0.8067870671570491,
 'F1 Score': 0.8710847833637567,
 'IoU Foreground': 0.771612226079539,
 'IoU Background': 0.9359531729525978,
 'Mean IoU': 0.8537826995160684}

In [5]:
metrics_from_dataset(crf_large,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cpu
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8385291022474306.pth


Processing images: 100%|██████████| 250/250 [01:09<00:00,  3.61it/s]


{'Foreground Acc': 0.8875778898283749,
 'Precision': 0.875033489966655,
 'Recall': 0.8875778898283749,
 'F1 Score': 0.8812610510254146,
 'IoU Foreground': 0.7877271563961918,
 'IoU Background': 0.9344301852468851,
 'Mean IoU': 0.8610786708215385}

In [ ]:
metrics_from_dataset_crf(crf_large,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

In [11]:
metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 250/250 [00:07<00:00, 32.00it/s]


{'Foreground Acc': 0.7660595847066114,
 'Precision': 0.9609540120555381,
 'Recall': 0.7660595847066114,
 'F1 Score': 0.8525098271114568,
 'IoU Foreground': 0.7429343163483996,
 'IoU Background': 0.9296760938585993,
 'Mean IoU': 0.8363052051034994}

In [4]:
metrics_from_dataset(crf_large,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8385291022474306.pth


Processing images: 100%|██████████| 4071/4071 [02:18<00:00, 29.41it/s]


{'Foreground Acc': 0.8531264479496747,
 'Precision': 0.8592444266360957,
 'Recall': 0.8531264479496747,
 'F1 Score': 0.8561745080999859,
 'IoU Foreground': 0.7485184708357829,
 'IoU Background': 0.9377373406950237,
 'Mean IoU': 0.8431279057654033}

In [5]:
metrics_from_dataset(crf_proper,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8696894799916581.pth


Processing images: 100%|██████████| 4071/4071 [02:07<00:00, 32.02it/s]


{'Foreground Acc': 0.8530342383267585,
 'Precision': 0.8995911819369873,
 'Recall': 0.8530342383267585,
 'F1 Score': 0.8756943381245789,
 'IoU Foreground': 0.7788756810704475,
 'IoU Background': 0.9473938138407835,
 'Mean IoU': 0.8631347474556155}

In [5]:
metrics_from_dataset(crf_small,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 4071/4071 [02:10<00:00, 31.18it/s]


{'Foreground Acc': 0.7814791465565297,
 'Precision': 0.9154443015948529,
 'Recall': 0.7814791465565297,
 'F1 Score': 0.8431737239646689,
 'IoU Foreground': 0.7288680603403905,
 'IoU Background': 0.9378197146343702,
 'Mean IoU': 0.8333438874873804}

In [16]:
metrics_from_dataset_crf(crf_small,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/base_sam_dino_india_tuned_freetown_eastafrica_haiti_0.8754172925254217.pth


Processing images: 100%|██████████| 250/250 [01:40<00:00,  2.49it/s]


{'Foreground Acc': 0.8067870671570491,
 'Precision': 0.9465185416001765,
 'Recall': 0.8067870671570491,
 'F1 Score': 0.8710847833637567,
 'IoU Foreground': 0.771612226079539,
 'IoU Background': 0.9359531729525978,
 'Mean IoU': 0.8537826995160684}

In [7]:
metrics_from_dataset_crf(crf_large,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/crf_sam_large_freetown_eastafrica_haiti_0.8385291022474306.pth


Processing images: 100%|██████████| 250/250 [01:46<00:00,  2.35it/s]


{'Foreground Acc': 0.8601681995385814,
 'Precision': 0.9201949540673127,
 'Recall': 0.8601681995385814,
 'F1 Score': 0.8891696452619142,
 'IoU Foreground': 0.8004549402789456,
 'IoU Background': 0.9416534532516262,
 'Mean IoU': 0.8710541967652858}

In [18]:
metrics_from_dataset(inhouse_default,'/teamspace/studios/this_studio/Deeplab/india_sam_dino_annotations_large')

Device: cuda
Resume model from /teamspace/studios/this_studio/Deeplab/saved_models/best_deeplabv3plus_mobilenet_custom_os16_0.7854892764326529.pth


Processing images: 100%|██████████| 4071/4071 [02:08<00:00, 31.73it/s]


{'Foreground Acc': 0.7355098986856355,
 'Precision': 0.9435659381416277,
 'Recall': 0.7355098986856355,
 'F1 Score': 0.8266476979111712,
 'IoU Foreground': 0.7045178983665468,
 'IoU Background': 0.9346595961708383,
 'Mean IoU': 0.8195887472686925}